This note shows how to use SABRE to transform a QuantumCircuit. The objective is either depth or SWAP count.
https://qiskit.org/documentation/tutorials/circuits_advanced/04_transpiler_passes_and_passmanager.html
"This is a design philosophy of Qiskit’s transpiler: every pass performs a small, well-defined action, and the aggressive circuit optimization is achieved by the pass manager through combining multiple passes."

Note we don't use SabreLayout directly; instead, we restructured it as sabre_mapper so that the seed can be easilly changed every time. A question: is the SabreSwap used in sabre_mapper has attribute fakerun=True?

## Issues with SabreSwap
0. The seed in sabre_mapper seems to play an important role.
1. Running for example "excitation_preserving_6.qasm" on grid2x3 obtains much better depth result than on tokyo!
2. Sabre is rather unstable on say "grover_operator_14".
3. SabreSwap is greatly affected by its initial mapping: for 'grover_operator_14', five runs may have results:
      Transformed circuit: swap count = 1321, final depth = 41189
      Transformed circuit: swap count = 2129, final depth = 41382
      Transformed circuit: swap count = 1066, final depth = 41113
      Transformed circuit: swap count = 951, final depth = 41863
      Transformed circuit: swap count = 848, final depth = 41190

In [4]:
import copy
import random
import numpy as np
from qiskit import QuantumCircuit
from qiskit.transpiler.layout import Layout
from qiskit.transpiler import CouplingMap
from qiskit.transpiler.passes.layout.set_layout import SetLayout
from qiskit.transpiler.passes.layout.full_ancilla_allocation import FullAncillaAllocation
from qiskit.transpiler.passes.layout.enlarge_with_ancilla import EnlargeWithAncilla
from qiskit.transpiler.passes.layout.apply_layout import ApplyLayout
from qiskit.transpiler.passes.routing import SabreSwap
from qiskit.transpiler.passmanager import PassManager

#from qiskit.converters import dag_to_circuit, circuit_to_dag
#from qiskit.dagcircuit import DAGCircuit, DAGOpNode, DAGInNode, DAGOutNode

In [57]:
#The following four functions are part of functions of SabreLayout
def _layout_and_route_passmanager(initial_layout, coupling_map):
    layout_and_route = [
            SetLayout(initial_layout),
            FullAncillaAllocation(coupling_map),
            EnlargeWithAncilla(),
            ApplyLayout(),
            SabreSwap(coupling_map, heuristic = 'lookahead'),
        ]
    pm = PassManager(layout_and_route)
    return pm

def _compose_layouts(initial_layout, pass_final_layout, qregs):
    trivial_layout = Layout.generate_trivial_layout(*qregs)
    qubit_map = Layout.combine_into_edge_map(initial_layout, trivial_layout)
    final_layout = {v: pass_final_layout._v2p[qubit_map[v]] for v in initial_layout._v2p}
    return Layout(final_layout)

"""Transform the input circuit and generate an initial layout"""
def sabre_mapper(cir_in: QuantumCircuit, coupling_map: CouplingMap):
    circ = cir_in.copy()
    #circ.draw('mpl')
    """everytime use a different seed"""
    seed = np.random.randint(0, np.iinfo(np.int32).max) 
    rng = np.random.default_rng(seed)

    physical_qubits = rng.choice(coupling_map.size(), len(circ.qubits), replace=False)
    physical_qubits = rng.permutation(physical_qubits)
    initial_layout = Layout({q: circ.qubits[i] for i, q in enumerate(physical_qubits)})
    #print(initial_layout)
    
    max_iterations = 3
    rev_circ = circ.reverse_ops()
    for i in range(max_iterations):
        for _ in ("forward", "backword"):
            pm = _layout_and_route_passmanager(initial_layout, coupling_map)
            new_circ = pm.run(circ)
            #new_circ.draw('mpl')
            pass_final_layout = pm.property_set["final_layout"]
            final_layout = _compose_layouts(initial_layout, pass_final_layout, new_circ.qregs)
            initial_layout = final_layout
            circ, rev_circ = rev_circ, circ
    return circ, initial_layout

"""One SABRE run with output = number of swaps"""            
def execute_sabre(circ: QuantumCircuit, coupling_map: CouplingMap, obj='swap_count'):
    newcirc, initial_layout = sabre_mapper(circ, coupling_map)
    #print(initial_layout)
    pm_final = _layout_and_route_passmanager(initial_layout, coupling_map)
    circ_final = pm_final.run(newcirc)
    print(f"  Transformed circuit: swap count = {circ_final.count_ops()['swap']}, final depth = {circ_final.depth()}")
    
    if obj == 'swap_count':
        num_swap = circ_final.count_ops()['swap']
        return num_swap  
    else: #obj == 'depth_count'        
        return circ_final.depth()

In [58]:
from ag import qgrid, q20
import time

path = '../bench/qiskit_circuit_benchmark/' 
#filename = 'excitation_preserving_6.qasm'
#filename = 'grover_operator_5.qasm'
#filename = 'quantum_volume_5.qasm'
#filename = 'phase_oracle_10.qasm'
#filename = 'OR_6.qasm'
filename = 'integer_comparator_12.qasm'

#AG = q20()
AG = qgrid(1,12)
print(AG.edges())
coupling_map = CouplingMap(AG.edges())
repeat = 100
obj = 'swap_count'

"""Create a QuantumCircuit from the QASM file"""
with open(path+filename, 'r') as file:
    qasm_code = file.read()
qc = QuantumCircuit.from_qasm_str(qasm_code)
print(f"{filename}: qubit number={qc.num_qubits}, input depth={qc.depth()}, \n  -----{qc.count_ops()}")

best_value = None
for i in range(repeat):
    num_swap = execute_sabre(qc, coupling_map, obj)
    if best_value == None or num_swap < best_value:
        best_value = num_swap
        
print(f'The best {obj} = {best_value} after {repeat} repeats')

[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11)]
integer_comparator_12.qasm: qubit number=12, input depth=93, 
  -----OrderedDict([('cx', 54), ('t', 36), ('tdg', 27), ('h', 18), ('u3', 5)])
  Transformed circuit: swap count = 18, final depth = 117
  Transformed circuit: swap count = 18, final depth = 117
  Transformed circuit: swap count = 13, final depth = 116
  Transformed circuit: swap count = 26, final depth = 125
  Transformed circuit: swap count = 22, final depth = 121
  Transformed circuit: swap count = 26, final depth = 125
  Transformed circuit: swap count = 14, final depth = 113
  Transformed circuit: swap count = 30, final depth = 129
  Transformed circuit: swap count = 22, final depth = 121
  Transformed circuit: swap count = 16, final depth = 115
  Transformed circuit: swap count = 26, final depth = 125
  Transformed circuit: swap count = 14, final depth = 113
  Transformed circuit: swap count = 16, final depth = 115
  Transformed